In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib as plt
import plotly.express as px
from statsmodels.stats.proportion import proportions_ztest


In [12]:
conv_data=pd.read_csv('/Users/danielpavelko/Downloads/application_events.csv')
conv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74359 entries, 0 to 74358
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   u_id          74359 non-null  object 
 1   step_2        74359 non-null  int64  
 2   step_3        74359 non-null  int64  
 3   step_4        74359 non-null  int64  
 4   step_5        74359 non-null  int64  
 5   step_6        74359 non-null  int64  
 6   step_7        74359 non-null  int64  
 7   step_8        74359 non-null  int64  
 8   flag_int      65686 non-null  float64
 9   flag_avia_tr  74359 non-null  int64  
 10  fork          74359 non-null  object 
 11  date          74359 non-null  object 
dtypes: float64(1), int64(8), object(3)
memory usage: 6.8+ MB


# Open and view data

In [13]:
def check_data(data):
    """The function performs general analysis of dataset info, describe,types. Search for omissions and duplicates.
    Check of data types."""
    isnull_dict = {}
    not_lower_cols = []
    count_cap_cols = 0
    display(data.info(),'Data types',data.dtypes, data.describe())
    for value in data.columns:
        isnull_dict[value] = data[value].isnull().sum()
        tmp_df = pd.DataFrame([isnull_dict])
        if value.islower():
            continue
        else:
            not_lower_cols.append(value)
            count_cap_cols += 1
    tmp_df.index = ['Column gaps']
    tmp_df = tmp_df.style.applymap(lambda x: 'color:darkred' if x > 0 else 'color:green')
    display(tmp_df)
    print(f'Number of lines = {data.shape[0]}, number of columns = {data.shape[1]}\n')
    print('Total duplicate rows in the table =', data.duplicated().sum())
    if count_cap_cols:
        print(f'В {count_cap_cols} the columns have capital letters.')
        print('to lowercase', not_lower_cols)
    else:
        print('do not need to be renamed')
    return data.head(10)

In [14]:
check_data(conv_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74359 entries, 0 to 74358
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   u_id          74359 non-null  object 
 1   step_2        74359 non-null  int64  
 2   step_3        74359 non-null  int64  
 3   step_4        74359 non-null  int64  
 4   step_5        74359 non-null  int64  
 5   step_6        74359 non-null  int64  
 6   step_7        74359 non-null  int64  
 7   step_8        74359 non-null  int64  
 8   flag_int      65686 non-null  float64
 9   flag_avia_tr  74359 non-null  int64  
 10  fork          74359 non-null  object 
 11  date          74359 non-null  object 
dtypes: float64(1), int64(8), object(3)
memory usage: 6.8+ MB


None

'Data types'

u_id             object
step_2            int64
step_3            int64
step_4            int64
step_5            int64
step_6            int64
step_7            int64
step_8            int64
flag_int        float64
flag_avia_tr      int64
fork             object
date             object
dtype: object

,step_2,step_3,step_4,step_5,step_6,step_7,step_8,flag_int,flag_avia_tr
count,74359.000000,74359.000000,74359.000000,74359.00000,74359.000000,74359.000000,74359.000000,65686.000000,74359.000000
mean,0.885071,0.883363,0.340107,0.12870,0.096397,0.066233,0.047661,0.270529,0.336126
std,0.318938,0.320989,0.473748,0.33487,0.295137,0.248690,0.213049,0.444237,0.472386
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


,u_id,step_2,step_3,step_4,step_5,step_6,step_7,step_8,flag_int,flag_avia_tr,fork,date
Column gaps,0,0,0,0,0,0,0,0,8673,0,0,0


Number of lines = 74359, number of columns = 12

Total duplicate rows in the table = 27
do not need to be renamed


,u_id,step_2,step_3,step_4,step_5,step_6,step_7,step_8,flag_int,flag_avia_tr,fork,date
0,5e1f9da200713a7ff20753070e39880c,1,1,0,0,0,0,0,1.0,0,control,2022-07-03
1,642c1e7c5c07733ddc8b0a83f39edb50,1,1,1,1,0,0,0,1.0,0,control,2022-07-03
2,f875555ea6fbfe430cee51a8d3088b5d,1,1,1,0,0,0,0,0.0,1,control,2022-07-03
3,4f6f9c26128a1b6ec99c9f9ac512b258,1,1,0,0,0,0,0,0.0,1,control,2022-07-03
4,cbe4d371e6e0f7f0f96ca310dcc9bc47,1,1,1,0,0,0,0,0.0,1,control,2022-07-03
5,2416982ef950c60497236eb5275c04d8,1,1,0,0,0,0,0,0.0,1,control,2022-07-03
6,ebd978e704d5607010f6fde6b84468f9,1,1,1,1,1,0,0,1.0,1,control,2022-07-03
7,78d721b1bb54ceba4b56474b18609dac,1,1,0,0,0,0,0,0.0,0,control,2022-07-03
8,2cffe0af33d6eb61d6a50e5a66c584a2,1,1,1,0,0,0,0,1.0,0,control,2022-07-03
9,e2473c7af379185eb69218e61cd656e8,1,1,1,1,1,1,1,0.0,0,control,2022-07-03


In [23]:
conv_data['step_1'] = 1
to_melt = ['u_id','step_1','step_2','step_3','step_4','step_5','step_6','step_7','step_8']
to_melt_data = conv_data[to_melt]
user_events = pd.melt(to_melt_data, id_vars=['u_id'], var_name='event', value_name='value')
#upv = upv.query('flag_avia_tr == 1')
user_events.head(15)
user_events = user_events.query('value == 1')
user_events = user_events.merge(conv_data, on='u_id', how='left')
user_events.head()

,u_id,event,value,step_2,step_3,step_4,step_5,step_6,step_7,step_8,flag_int,flag_avia_tr,fork,date,step_1
0,5e1f9da200713a7ff20753070e39880c,step_1,1,1,1,0,0,0,0,0,1.0,0,control,2022-07-03,1
1,642c1e7c5c07733ddc8b0a83f39edb50,step_1,1,1,1,1,1,0,0,0,1.0,0,control,2022-07-03,1
2,f875555ea6fbfe430cee51a8d3088b5d,step_1,1,1,1,1,0,0,0,0,0.0,1,control,2022-07-03,1
3,4f6f9c26128a1b6ec99c9f9ac512b258,step_1,1,1,1,0,0,0,0,0,0.0,1,control,2022-07-03,1
4,cbe4d371e6e0f7f0f96ca310dcc9bc47,step_1,1,1,1,1,0,0,0,0,0.0,1,control,2022-07-03,1


In [36]:
user_events_basic = user_events[['u_id','event','value','fork']]

In [80]:
#events = list(user_events['event'].dropna().unique())
events = ['step_2','step_3','step_4','step_5','step_6','step_7','step_8']

def check_z(data, events, alpha):
    """Function for hypothesis testing using z_test
    Accepted arguments data - dataset, events - conversion events, alpha - stat significance level.
    The function counts conversion from the total number of users to those who made an event from events.
    """
    count = data.groupby('fork').agg({'u_id':'nunique'}).reset_index()
    count.columns = ['fork', 'total_users']
    for event in events:
        nobs = data[(data['event'] == event) & (data['fork'].isin(count['fork']))].groupby('fork').agg({
            'u_id':'nunique'}).reset_index()
        nobs.columns = ['fork', event]
        pivot_z = pd.merge(count, nobs, on = 'fork')
        pivot_z['conversions'] = round(pivot_z[event] / pivot_z['total_users'], 2)

        stat, p_value = proportions_ztest(pivot_z[event].values,pivot_z['total_users'].values)

        display(pivot_z)
        if p_value >= alpha:
            print('p value = {0:.10f}'.format(p_value))
            print(f'The null hypothesis is not rejected: there are no differences in event conversion between groups in {event}'.upper())
        else:
            print('p value = {0:.10f}'.format(p_value))
            print(f'The null hypothesis is rejected: there are differences in event conversion between groups in {event}'.upper())
    return 'test calculated'

In [81]:
check_z(user_events_basic, events, 0.05)

,fork,total_users,step_2,conversions
0,control,37111,32781,0.88
1,last-seats,37221,33012,0.89


p value = 0.1242534615
THE NULL HYPOTHESIS IS NOT REJECTED: THERE ARE NO DIFFERENCES IN EVENT CONVERSION BETWEEN GROUPS IN STEP_2


,fork,total_users,step_3,conversions
0,control,37111,32708,0.88
1,last-seats,37221,32958,0.89


p value = 0.0807012473
THE NULL HYPOTHESIS IS NOT REJECTED: THERE ARE NO DIFFERENCES IN EVENT CONVERSION BETWEEN GROUPS IN STEP_3


,fork,total_users,step_4,conversions
0,control,37111,12666,0.34
1,last-seats,37221,12619,0.34


...

,fork,total_users,step_7,conversions
0,control,37111,2399,0.06
1,last-seats,37221,2524,0.07


p value = 0.0825309032
THE NULL HYPOTHESIS IS NOT REJECTED: THERE ARE NO DIFFERENCES IN EVENT CONVERSION BETWEEN GROUPS IN STEP_7


,fork,total_users,step_8,conversions
0,control,37111,1718,0.05
1,last-seats,37221,1825,0.05


p value = 0.0798087094
THE NULL HYPOTHESIS IS NOT REJECTED: THERE ARE NO DIFFERENCES IN EVENT CONVERSION BETWEEN GROUPS IN STEP_8


'test calculated'

## Test Results

Since no significant difference was found in the total sample, it is necessary to test the funnel on international users only

In [100]:
user_events_int = user_events[user_events['flag_int'] == 1]
user_events_int = user_events_int[['u_id','event','value','fork','flag_avia_tr']]
user_events_int.head()

,u_id,event,value,fork,flag_avia_tr
0,5e1f9da200713a7ff20753070e39880c,step_1,1,control,0
1,642c1e7c5c07733ddc8b0a83f39edb50,step_1,1,control,0
6,ebd978e704d5607010f6fde6b84468f9,step_1,1,control,1
8,2cffe0af33d6eb61d6a50e5a66c584a2,step_1,1,control,0
11,cde00f937ef28186797de4fea5cbc5a1,step_1,1,control,1


In [99]:
check_z(user_events_int, events, 0.05)

/Users/danielpavelko/anaconda3/lib/python3.9/site-packages/statsmodels/stats/weightstats.py:790: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std


,fork,total_users,step_2,conversions
0,control,8866,8866,1.0
1,last-seats,8903,8903,1.0


p value = nan
THE NULL HYPOTHESIS IS REJECTED: THERE ARE DIFFERENCES IN EVENT CONVERSION BETWEEN GROUPS IN STEP_2


/Users/danielpavelko/anaconda3/lib/python3.9/site-packages/statsmodels/stats/weightstats.py:790: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std


,fork,total_users,step_3,conversions
0,control,8866,8866,1.0
1,last-seats,8903,8903,1.0


...

,fork,total_users,step_7,conversions
0,control,8866,433,0.05
1,last-seats,8903,447,0.05


p value = 0.6739590399
THE NULL HYPOTHESIS IS NOT REJECTED: THERE ARE NO DIFFERENCES IN EVENT CONVERSION BETWEEN GROUPS IN STEP_7


,fork,total_users,step_8,conversions
0,control,8866,273,0.03
1,last-seats,8903,270,0.03


p value = 0.8571235958
THE NULL HYPOTHESIS IS NOT REJECTED: THERE ARE NO DIFFERENCES IN EVENT CONVERSION BETWEEN GROUPS IN STEP_8


'test calculated'

## Overall Result

According to the calculations in step 4, we get a statistically significant conversion difference, and decide not to roll out the update